In [36]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib

import lightgbm as lgb
from lightgbm import LGBMClassifier

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from sklearn.metrics import log_loss, make_scorer
from functools import partial

from sklearn.model_selection import train_test_split

import gc
import time
import warnings
warnings.filterwarnings('ignore')

In [37]:
train = pd.read_csv('../dataset/train20200225.csv', index_col = 0)
test = pd.read_csv('../dataset/test20200225.csv', index_col = 0)
sample_submission = pd.read_csv('../dataset/sample_submission.csv', index_col = 0)

In [38]:
column_number = {}

for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [39]:
#bins= [0, 100, 200, 300, 400, 500, 600, 700, 800, 900 ,1000]
#labels = ['0', '1','2','3','4','5','6','7','8','9']

#train['fiberID2'] = pd.cut(train['fiberID'], bins= bins, labels = labels)
#test['fiberID2'] = pd.cut(test['fiberID'], bins= bins, labels = labels)

In [40]:
train_x = train.drop(columns = ['type', 'type_num'], axis = 1)
train_y = train['type_num']

test_x = test

In [41]:
train_x.drop(columns = 'fiberID', inplace = True)
test_x.drop(columns = 'fiberID', inplace = True)

In [42]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe_train = pd.DataFrame(ohe.fit_transform(train_x[['fiberID2']]).toarray(),
                         columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_test = pd.DataFrame(ohe.fit_transform(test_x[['fiberID2']]).toarray(),
                        columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_train.head()

,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
idx_list = [x for x in range(199991,210000)]
ohe_test['id'] = idx_list
ohe_test.set_index('id', inplace = True)

In [44]:
train_x = pd.concat([train_x, ohe_train], axis = 1)
test_x = pd.concat([test_x, ohe_test], axis = 1)
train_x.drop('fiberID2', axis = 1, inplace = True)
test_x.drop('fiberID2', axis = 1, inplace = True)
train_x.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,...,petroMag_i_z,modelMag_i_z,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,...,0.722964,-0.126735,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,...,-0.006847,0.181074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,...,-0.063796,-0.028751,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,...,0.295800,0.205258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,...,1.233871,1.111675,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
#train_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)
#test_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)

# Modeling

In [46]:
space = {'objective' : 'multiclass',
         'num_class' : 19,
              'metric' : 'multi_logloss',
              'boosting' : 'gbdt',
    'max_depth':  hp.choice('max_depth', list(range(4, 10, 2))),
    'reg_alpha' : hp.uniform('reg_alpha', 0.01, 0.4),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.4),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 0.9),
    'gamma' : hp.uniform('gamma', 0.01, .7),
    'num_leaves' : hp.choice('num_leaves', list(range(20,150,10))),
    'min_child_samples': hp.choice('min_child_samples', list(range(100,150,10))),
    'subsample': hp.choice('subsample', [.2, .4, .5, .6, .7, .8, .9]),
    'feature_fraction' : hp.uniform('feature_fraction', .4, .8),
    'bagging_fraction' : hp.uniform('bagging_fraction', .7, .9)
}

In [47]:
def evaluate_metric(params):
    time1 = time.time()
    print("******************** New Run *********************")
    print('Params : {}'.format(params))
    FOLDS = 5
    count = 1
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = 42)
    y_preds = np.zeros(test.shape[0])
    score_mean = 0
    
    for tr_idx, val_idx in skf.split(train_x, train_y):
        X_tr, X_vl = train_x.iloc[tr_idx, :], train_x.iloc[val_idx, :]
        y_tr, y_vl = train_y.iloc[tr_idx], train_y.iloc[val_idx]
        
        lgbtrain = lgb.Dataset(X_tr, label = y_tr)
        lgbval = lgb.Dataset(X_vl, label = y_vl)
        
        lgb_clf = lgb.train(params, lgbtrain, 1250, valid_sets = [lgbval],
                           categorical_feature = 'auto',
                           verbose_eval = 0,
                           early_stopping_rounds = 50)
        
        lgb_pred = lgb_clf.predict(X_vl, num_iteration = lgb_clf.best_iteration)
        
        score = make_scorer(log_loss)(lgb_clf, X_vl, y_vl)
        
        score_mean += score
        print('{} CV - Score: {}'.format(count, round(score,4)))
        
        count += 1
    
    time2 = time.time() - time1
    print('Total Time Run : {}'.format(round(time2/60, 2)))
    gc.collect()
    print('Mean Log Loss : {}'.format(score_mean/FOLDS))
    del X_tr, X_vl, y_tr, y_vl, lgb_clf, score
    
    return (score_mean/FOLDS)

In [48]:
best = fmin(fn = evaluate_metric,
            space = space,
            algo = tpe.suggest,
            max_evals = 3)

best_params = space_eval(space,best)
print("Best HyperParameters :", best_params)

******************** New Run *********************                                                                     
Params : {'bagging_fraction': 0.43205555784747446, 'boosting': 'gbdt', 'colsample_bytree': 0.6776154916928419, 'feature_fraction': 0.7883174382200635, 'gamma': 0.6750134426114474, 'learning_rate': 0.12620727419679306, 'max_depth': 6, 'metric': 'multi_logloss', 'min_child_samples': 110, 'num_class': 19, 'num_leaves': 100, 'objective': 'multiclass', 'reg_alpha': 0.0579170679090923, 'reg_lambda': 0.10097176964316099, 'subsample': 0.9}
1 CV - Score: 0.3679                                                                                                   
2 CV - Score: 0.3684                                                                                                   
3 CV - Score: 0.383                                                                                                    
4 CV - Score: 0.37                                                                  

Best HyperParameters : {'bagging_fraction': 0.4778711498115298, 'boosting': 'gbdt', 'colsample_bytree': 0.530825962221535, 'feature_fraction': 0.6685108367216006, 'gamma': 0.3089418643344476, 'learning_rate': 0.037576672680065454, 'max_depth': 8, 'metric': 'multi_logloss', 'min_child_samples': 100, 'num_class': 19, 'num_leaves': 140, 'objective': 'multiclass', 'reg_alpha': 0.15641799752802943, 'reg_lambda': 0.31890794153928625, 'subsample': 0.7}

In [74]:
best_params = {'bagging_fraction': 0.6778711498115298, 'boosting': 'gbdt', 'colsample_bytree': 0.530825962221535, 'feature_fraction': 0.6685108367216006, 'gamma': 0.3089418643344476, 'learning_rate': 0.037576672680065454, 'max_depth': 6, 'metric': 'multi_logloss', 'min_child_samples': 100, 'num_class': 19, 'num_leaves': 31, 'objective': 'multiclass', 'reg_alpha': 0.15641799752802943, 'reg_lambda': 0.31890794153928625, 'subsample': 0.7}

In [81]:
best_params = {'bagging_fraction': 0.4778711498115298, 'boosting': 'gbdt', 'colsample_bytree': 0.530825962221535, 'feature_fraction': 0.6685108367216006, 'gamma': 0.3089418643344476, 'learning_rate': 0.037576672680065454, 'max_depth': 8, 'metric': 'multi_logloss', 'min_child_samples': 100, 'num_class': 19, 'num_leaves': 140, 'objective': 'multiclass', 'reg_alpha': 0.15641799752802943, 'reg_lambda': 0.31890794153928625, 'subsample': 0.7}

In [83]:
X_tr, X_vl, y_tr, y_vl = train_test_split(train_x, train_y, random_state = 42, shuffle = True)

X_test = test_x

lgb_train = lgb.Dataset(X_tr, label = y_tr)
lgb_valid = lgb.Dataset(X_vl, label = y_vl)

lgb_model = lgb.train(best_params,
                      lgb_train, 1250,
                      valid_sets = [lgb_train, lgb_valid],
                      verbose_eval = 10,
                      categorical_feature = 'auto',
                      early_stopping_rounds = 50)

Training until validation scores don't improve for 50 rounds
[10]	training's multi_logloss: 1.39613	valid_1's multi_logloss: 1.41304
[20]	training's multi_logloss: 1.02512	valid_1's multi_logloss: 1.04895
[30]	training's multi_logloss: 0.80245	valid_1's multi_logloss: 0.831504
[40]	training's multi_logloss: 0.656707	valid_1's multi_logloss: 0.690001
[50]	training's multi_logloss: 0.5579	valid_1's multi_logloss: 0.594894
[60]	training's multi_logloss: 0.489195	valid_1's multi_logloss: 0.529678
[70]	training's multi_logloss: 0.440269	valid_1's multi_logloss: 0.483769
[80]	training's multi_logloss: 0.404607	valid_1's multi_logloss: 0.451142
[90]	training's multi_logloss: 0.378045	valid_1's multi_logloss: 0.427597
[100]	training's multi_logloss: 0.357793	valid_1's multi_logloss: 0.410292
[110]	training's multi_logloss: 0.342053	valid_1's multi_logloss: 0.397416
[120]	training's multi_logloss: 0.329733	valid_1's multi_logloss: 0.387708
[130]	training's multi_logloss: 0.319417	valid_1's mult

In [52]:
import time
time = time.gmtime(time.time()) 
year = time[0]
month = time[1]
day = time[2]

joblib.dump(lgb_model, '../model/LGB_OPT_3845_{}{}{}_3479.pkl'.format(year, month, day))

['../model/LGB_OPT_3845_2020225_3479.pkl']

In [53]:
pred = lgb_model.predict(test_x, num_iteration = lgb_model.best_iteration)

In [54]:
submission = pd.DataFrame(data = pred, columns = sample_submission.columns, index = sample_submission.index)

In [55]:
submission.to_csv('../submission/lgb_opt_3479.csv')